# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Makes decision on which items to include in menu.'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [3]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Makes decisions on which items to include in Menu'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 1),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([dish_price])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: No subtasks have been completed yet.
Thoughts: To complete the assigned task, we need to generate 5 menu items with name, description, ingredients, and price based on Italian food choices. We can start by using the Creative Writer function to generate interesting dish names and descriptions for Italian cuisine.
Subtask identified: Use the Creative Writer function to generate interesting dish names and descriptions for Italian cuisine.
Calling function Creative Writer with parameters {'instruction': 'Use the Creative Writer function to generate interesting dish names and descriptions for Italian cuisine.'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet.
Thoughts: To generate interesting dish names and descriptions for Italian cuisine, I can use the Creative Writer function.
Subtask identified: Use the Creative Writer function to generate interesting dish names and descriptions for Italian cuisine.
Getting LLM to perform the f

In [9]:
output = my_agent.reply_user()

{'Menu': [{'Name': 'Bella Napoli Risotto', 'Description': 'Creamy arborio rice cooked with saffron, parmesan cheese, and a medley of fresh seafood.', 'Ingredients': 'Arborio rice, saffron, parmesan cheese, seafood (such as shrimp, mussels, and calamari)', 'Price': '$16.99'}, {'Name': 'Amore Caprese Salad', 'Description': 'Ripe tomatoes, fresh basil, and creamy mozzarella drizzled with balsamic glaze and extra virgin olive oil.', 'Ingredients': 'Tomatoes, fresh basil, mozzarella cheese, balsamic glaze, extra virgin olive oil', 'Price': '$9.99'}, {'Name': 'Vesuvio Pasta Carbonara', 'Description': 'Spaghetti tossed in a rich egg and pancetta sauce, finished with a sprinkle of black pepper and pecorino romano cheese.', 'Ingredients': 'Spaghetti, eggs, pancetta, black pepper, pecorino romano cheese', 'Price': '$14.99'}, {'Name': 'Dolce Vita Tiramisu', 'Description': 'Layers of espresso-soaked ladyfingers and mascarpone cream dusted with cocoa powder, a classic Italian dessert.', 'Ingredient

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Use the Creative Writer function to generate interesting dish names and descriptions for Italian cuisine.
{'Agent Output': 'As an agent specializing in generating interesting dish names and descriptions for Italian cuisine, I have come up with the following creations: 1. Bella Napoli Risotto - Creamy arborio rice cooked with saffron, parmesan cheese, and a medley of fresh seafood. 2. Amore Caprese Salad - Ripe tomatoes, fresh basil, and creamy mozzarella drizzled with balsamic glaze and extra virgin olive oil. 3. Vesuvio Pasta Carbonara - Spaghetti tossed in a rich egg and pance

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet.
Thoughts: To complete the assigned task of generating 5 Italian dish names and descriptions, I can start by using a language model to generate creative and unique names and descriptions for Italian dishes.
Subtask identified: Generate Italian dish names and descriptions using a language model.
Getting LLM to perform the following task: Generate Italian dish names and descriptions using a language model.
> As an agent specializing in generating dish names and descriptions, I have used a language model to create a variety of Italian dish names and descriptions. Here are some examples: 1. Risotto al Funghi: A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese. 2. Pollo alla Cacciatora: Tender chicken cooked in a rich tomato and herb sauce, served 

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Generate only 5 Italian dish names and descriptions
{'Agent Output': 'As an agent specializing in generating dish names and descriptions, I have used a language model to create a variety of Italian dish names and descriptions. Here are some examples: 1. Risotto al Funghi: A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese. 2. Pollo alla Cacciatora: Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables. 3. Cannoli Siciliani: Crispy pastry tubes filled with sweet ricotta cheese and chocolate chips, dusted with powdered

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: No subtasks have been completed for the Assigned Task
Thoughts: Generate ingredients for the 5 Italian dishes based on the dish names and descriptions provided earlier
Subtask identified: Use the LLM function to generate ingredients for the 5 Italian dishes
Getting LLM to perform the following task: Use the LLM function to generate ingredients for the 5 Italian dishes
> As an agent specializing in generating dish names and descriptions, I have used a language model to create a variety of Italian dish names and descriptions. Here are the corresponding ingredients for the Italian dishes: 1. Risotto al Funghi: Arborio rice, mushrooms, Parmesan cheese, onion, garlic, white wine, butter, chicken or vegetable broth. 2. Pollo alla Cacciatora: Chicken pieces, tomatoes, bell peppers, onion, garlic, herbs (such as rosemary and thyme), olives, cap

In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: No subtasks have been completed for the Assigned Task
Thoughts: Need to generate prices for the 5 Italian dishes
Subtask identified: Use the dish_price function to generate prices for the 5 Italian dishes
Calling function dish_price with parameters {'list_of_dish_names': ['Risotto al Funghi', 'Pollo alla Cacciatora', 'Cannoli Siciliani', 'Tagliatelle al Tartufo', 'Tiramisu Classico']}
> {'output_1': {'Risotto al Funghi': '$10', 'Pollo alla Cacciatora': '$6', 'Cannoli Siciliani': '$9', 'Tagliatelle al Tartufo': '$10', 'Tiramisu Classico': '$2'}}

Observation: The dish prices for the 5 Italian dishes have been successfully generated using the dish_price function.
Thoughts: To complete the remainder of the Assigned Task, we need to provide the pricing information to the user.
Subtask identified: End Task
Task completed successfully!



In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: No subtasks have been completed for the Assigned Task
Thoughts: Based on the task requirements, the next step is to select 3 Italian dishes that will please Gordon Ramsay
Subtask identified: Choose 3 Italian dishes that will please Gordon Ramsay
Getting LLM to perform the following task: Choose 3 Italian dishes that will please Gordon Ramsay
> Based on Gordon Ramsay's known preferences for bold flavors, high-quality ingredients, and expertly executed dishes, I recommend the following 3 Italian dishes that are likely to please him: 1. Risotto al Funghi: The creamy and flavorful risotto made with fresh mushrooms and Parmesan cheese showcases a balance of rich umami flavors that Gordon Ramsay would appreciate. 2. Pollo alla Cacciatora: The tender chicken cooked in a rich tomato and herb sauce, served with roasted veg

In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Risotto al Funghi, A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese, Ingredients: Arborio rice, mushrooms, Parmesan cheese, onion, garlic, white wine, butter, chicken or vegetable broth, Price: $10
Pollo alla Cacciatora, Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables, Ingredients: Chicken pieces, tomatoes, bell peppers, onion, garlic, herbs (such as rosemary and thyme), olives, capers, red wine, olive oil, Price: $6
Tagliatelle al Tartufo, Thin ribbon pasta tossed in a decadent truffle cream sauce, garnished with fresh parsley, Ingredients: Tagliatelle pasta, heavy cream, truffle oil, Parmesan cheese, garlic, shallots, parsley, salt, pepper, Price: $10


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Generate only 5 Italian dish names and descriptions
{'Agent Output': 'As an agent specializing in generating dish names and descriptions, I have used a language model to create a variety of Italian dish names and descriptions. Here are some examples: 1. Risotto al Funghi: A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese. 2. Pollo alla Cacciatora: Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables. 3. Cannoli Siciliani: Crispy pastry tubes filled with sweet ricotta cheese and chocolate chips, dusted with powdered

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Risotto al Funghi, A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese, Ingredients: Arborio rice, mushrooms, Parmesan cheese, onion, garlic, white wine, butter, chicken or vegetable broth, Price: $10
Pollo alla Cacciatora, Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables, Ingredients: Chicken pieces, tomatoes, bell peppers, onion, garlic, herbs (such as rosemary and thyme), olives, capers, red wine, olive oil, Price: $6
Tagliatelle al Tartufo, Thin ribbon pasta tossed in a decadent truffle cream sauce, garnished with fresh parsley, Ingredients: Tagliatelle pasta, heavy cream, truffle oil, Parmesan cheese, garlic, shallots, parsley, salt, pepper, Price: $10


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
Risotto al Funghi, A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese, Ingredients: Arborio rice, mushrooms, Parmesan cheese, onion, garlic, white wine, butter, chicken or vegetable broth, Price: $10
Pollo alla Cacciatora, Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables, Ingredients: Chicken pieces, tomatoes, bell peppers, onion, garlic, herbs (such as rosemary and thyme), olives, capers, red wine, olive oil, Price: $6
Tagliate

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", 'Risotto al Funghi, A creamy and flavorful risotto dish made with fresh mushrooms and Parmesan cheese, Ingredients: Arborio rice, mushrooms, Parmesan cheese, onion, garlic, white wine, butter, chicken or vegetable broth, Price: $10\nPollo alla Cacciatora, Tender chicken cooked in a rich tomato and herb sauce, served with roasted vegetables, Ingredients: Chicken pieces, tomatoes, bell peppers, onion, garlic, herbs (such as rosemary and thyme), olives, capers, red wine, olive oil, Price: $6\nTagliatelle al Tartufo, Thin ribbon pasta tossed in a decadent truffle cream sauce, garnished with fresh parsley, Ingredients: Tagliatelle pasta, heavy cream, truffle oil, Parmesan cheese, garlic, shallots, parsley, salt, pepper, Price: $10')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False
